In [1]:
# dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from random import randint
from time import sleep
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
import os
from dotenv import load_dotenv

### Scraping from Billboard

In [2]:
# finding today's date
import datetime as dt
from datetime import date, timedelta, datetime
today = date.today()
today

datetime.date(2021, 5, 26)

In [15]:
dateTimeString = dt.datetime.strptime("01-04-2021", "%m-%d-%Y")

In [16]:
dateTimeString

datetime.datetime(2021, 1, 4, 0, 0)

In [17]:
justDateString = dateTimeString.date()
justDateString

datetime.date(2021, 1, 4)

In [18]:
type(justDateString)

datetime.date

In [21]:
# finding the gap between today's date and the last day for which scraping was done
num_days_since = int(str(today - justDateString).split(' ')[0])
# billboard charts come out every week,
# it is necessary to find number of weeks to know how many pages to scrape
num_weeks_since = round(num_days_since/7)
date_to_start = str(justDateString)
print(f'days: {num_days_since}, in weeks: {num_weeks_since}')
print(date_to_start)

days: 142, in weeks: 20
2021-01-04


In [22]:
# set up url based on last date scraped
billboard_url = f'https://www.billboard.com/charts/Hot-100/{date_to_start}'
print(billboard_url)

https://www.billboard.com/charts/Hot-100/2021-01-04


In [23]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/sarahdalleyhood/.wdm/drivers/chromedriver/mac64/90.0.4430.24]


In [24]:
# visit the site
browser.visit(billboard_url)
#sleep(randint(3,10))

In [25]:
# scraping billboard song info
counter = 0
billboard_info_list = []
# going for range of weeks inbetween last scraped day and present day
for counter in range(0, num_weeks_since):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    date_section = soup.find('div', class_='chart-detail-header__select-date')
    span_section = date_section.find('span')
    button = span_section.find('button')
    this_date = button.text
    print(this_date)
    
    for item in soup.find_all('div', class_='chart-details__left-rail'):
        list_item = item.find_all('div', class_='chart-list-item')
        for s in list_item:
            # current place on charts
            list_rank = s.get('data-rank')
            # performer
            list_artist = s.get('data-artist')
            # song name
            list_title = s.get('data-title')
            # getting previous stats
            list_extra = s.find('div', class_='chart-list-item__first-row')  
            ministats = list_extra.find_all('div', class_='chart-list-item__ministats-cell')
            for c in ministats:
                # last week's place on charts
                if c.find('span', class_='chart-list-item__ministats-cell-heading').text == 'Last':
                    lastWeek = c.text
                    lastWeekCleaned = lastWeek.strip().split('L')[0].strip()
                # peak of the song
                if c.find('span', class_='chart-list-item__ministats-cell-heading').text == 'Peak':
                    songPeak = c.text
                    songPeakCleaned = songPeak.strip().split('P')[0].strip()
                # how long the song has been on the chart
                if c.find('span', class_='chart-list-item__ministats-cell-heading').text == 'Weeks':
                    songWeeks = c.text
                    songWeeksCleaned = songWeeks.strip().split('W')[0].strip()
            # appending to the data list
            billboard_info_list.append(
                {'date': this_date, 'song': list_title, 'performer': list_artist, 'chart_position': list_rank, 
                'previous_position': lastWeekCleaned, 'peak': songPeakCleaned, 'weeks_on_chart': songWeeksCleaned,
                'hitTF': 1})  
    # clicking on the dropdown to get to the next week link
    for l in span_section.find('label').find('ul').find_all('a'):
        if 'Next Week' in l.text:
            partial_link = l['href']
            
    
    # open the menu to get the next week
    browser.find_by_css('span.chart-detail-header__date-selector').first.click()
    
    # ensuring that if it is the last week's data to be collected, it won't try finding 'next week' option
    if counter != num_weeks_since - 1:
        # clicking on the link to the next week
        try:
            print(partial_link)
            browser.click_link_by_partial_href(partial_link)
            sleep(randint(3,10))
        except AttributeError as e:
            print(e)
    
#     increment the counter
    counter +=1


January 9, 2021


/charts/Hot-100/2021-01-16


/Users/sarahdalleyhood/~Applications/lib/python3.8/site-packages/splinter/driver/webdriver/__init__.py:482: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  warnings.warn(



January 16, 2021


/charts/Hot-100/2021-01-23

January 23, 2021


/charts/Hot-100/2021-01-30

January 30, 2021


/charts/Hot-100/2021-02-06

February 6, 2021


/charts/Hot-100/2021-02-13

February 13, 2021


/charts/Hot-100/2021-02-20

February 20, 2021


/charts/Hot-100/2021-02-27

February 27, 2021


/charts/Hot-100/2021-03-06

March 6, 2021


/charts/Hot-100/2021-03-13

March 13, 2021


/charts/Hot-100/2021-03-20

March 20, 2021


/charts/Hot-100/2021-03-27

March 27, 2021


/charts/Hot-100/2021-04-03

April 3, 2021


/charts/Hot-100/2021-04-10

April 10, 2021


/charts/Hot-100/2021-04-17

April 17, 2021


/charts/Hot-100/2021-04-24

April 24, 2021


/charts/Hot-100/2021-05-01

May 1, 2021


/charts/Hot-100/2021-05-08

May 8, 2021


/charts/Hot-100/2021-05-15

May 15, 2021


/charts/Hot-100/2021-05-22

May 22, 2021




In [26]:
# examine the list
billboard_info_list

[{'date': '\nJanuary 9, 2021\n\n',
  'song': 'Mood',
  'performer': '24kGoldn Featuring iann dior',
  'chart_position': '1',
  'previous_position': '7',
  'peak': '1',
  'weeks_on_chart': '21',
  'hitTF': 1},
 {'date': '\nJanuary 9, 2021\n\n',
  'song': 'Positions',
  'performer': 'Ariana Grande',
  'chart_position': '2',
  'previous_position': '14',
  'peak': '1',
  'weeks_on_chart': '10',
  'hitTF': 1},
 {'date': '\nJanuary 9, 2021\n\n',
  'song': 'Blinding Lights',
  'performer': 'The Weeknd',
  'chart_position': '3',
  'previous_position': '-',
  'peak': '1',
  'weeks_on_chart': '56',
  'hitTF': 1},
 {'date': '\nJanuary 9, 2021\n\n',
  'song': 'Holy',
  'performer': 'Justin Bieber Featuring Chance The Rapper',
  'chart_position': '4',
  'previous_position': '21',
  'peak': '3',
  'weeks_on_chart': '15',
  'hitTF': 1},
 {'date': '\nJanuary 9, 2021\n\n',
  'song': 'Dynamite',
  'performer': 'BTS',
  'chart_position': '5',
  'previous_position': '44',
  'peak': '1',
  'weeks_on_chart'

In [27]:
browser.quit()

In [28]:
# examine the data as a df
bb21_df = pd.DataFrame(billboard_info_list)
bb21_df.head()

,date,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,"\nJanuary 9, 2021\n\n",Mood,24kGoldn Featuring iann dior,1,7,1,21,1
1,"\nJanuary 9, 2021\n\n",Positions,Ariana Grande,2,14,1,10,1
2,"\nJanuary 9, 2021\n\n",Blinding Lights,The Weeknd,3,-,1,56,1
3,"\nJanuary 9, 2021\n\n",Holy,Justin Bieber Featuring Chance The Rapper,4,21,3,15,1
4,"\nJanuary 9, 2021\n\n",Dynamite,BTS,5,44,1,19,1


In [29]:
bb21_df.shape

(2000, 8)

In [30]:
# removing the /n from the dates
clean_dates = []
for date in bb21_df['date']:
    clean_date = date.strip()
    clean_dates.append(clean_date)
clean_dates[0:5]

['January 9, 2021',
 'January 9, 2021',
 'January 9, 2021',
 'January 9, 2021',
 'January 9, 2021']

In [31]:
# adding the cleaned dates to the df
# bb21_df = bb21_df[['rank', 'artist', 'title']]
bb21_df['date'] = clean_dates
bb21_df.head()

,date,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,"January 9, 2021",Mood,24kGoldn Featuring iann dior,1,7,1,21,1
1,"January 9, 2021",Positions,Ariana Grande,2,14,1,10,1
2,"January 9, 2021",Blinding Lights,The Weeknd,3,-,1,56,1
3,"January 9, 2021",Holy,Justin Bieber Featuring Chance The Rapper,4,21,3,15,1
4,"January 9, 2021",Dynamite,BTS,5,44,1,19,1


In [32]:
# change date col to datetime
bb21_df['date'] = pd.to_datetime(bb21_df['date'])
bb21_df.head()

,date,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,2021-01-09,Mood,24kGoldn Featuring iann dior,1,7,1,21,1
1,2021-01-09,Positions,Ariana Grande,2,14,1,10,1
2,2021-01-09,Blinding Lights,The Weeknd,3,-,1,56,1
3,2021-01-09,Holy,Justin Bieber Featuring Chance The Rapper,4,21,3,15,1
4,2021-01-09,Dynamite,BTS,5,44,1,19,1


In [33]:
bb21_df.columns

Index(['date', 'song', 'performer', 'chart_position', 'previous_position',
       'peak', 'weeks_on_chart', 'hitTF'],
      dtype='object')

In [34]:
# initial shaping of the data for add on: removing date column
freshData = bb21_df[['song', 'performer', 'chart_position', 'previous_position', 'peak', 'weeks_on_chart', 'hitTF']]
freshData.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,Mood,24kGoldn Featuring iann dior,1,7,1,21,1
1,Positions,Ariana Grande,2,14,1,10,1
2,Blinding Lights,The Weeknd,3,-,1,56,1
3,Holy,Justin Bieber Featuring Chance The Rapper,4,21,3,15,1
4,Dynamite,BTS,5,44,1,19,1


### Getting Spotify track info

In [35]:
# additional imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [36]:
# preparing spotify credentials
load_dotenv(dotenv_path='spotifyCred.env')
CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')

In [37]:
# connecting to spotipy
spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
)

In [38]:
# getting only the song titles
song_titles_bb = freshData[['song', 'performer']]
song_titles_bb.loc[:, 'performer'] = song_titles_bb.loc[:, 'performer'].apply(lambda x: x.split('Featuring')[0])
song_titles_bb.loc[:, 'performer'] = song_titles_bb.loc[:, 'performer'].apply(lambda x: x.split('&')[0])
song_titles_bb

/Users/sarahdalleyhood/~Applications/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,song,performer
0,Mood,24kGoldn
1,Positions,Ariana Grande
2,Blinding Lights,The Weeknd
3,Holy,Justin Bieber
4,Dynamite,BTS
...,...,...
1995,Minimum Wage,Blake Shelton
1996,Richer,Rod Wave
1997,Drunk (And I Don't Wanna Go Home),Elle King
1998,Headshot,"Lil Tjay, Polo G"


In [39]:
# function to get spotify uris for each 2021 song
def get_song_uri(df, col_with_song_name):
    song_list = {
        'artist': [],
        'song': [],
        'spotifyName': [],
        'uri': []
    }

    for i, row in df.iterrows():
        song = row[f'{col_with_song_name}']
    #     artist_from_df = row['performer'].lower()
        results = spotify.search(q=song, limit=5)['tracks']['items']
        for track in results:
            song_list['artist'].append(track['artists'][0]['name'])
            song_list['song'].append(row[f'{col_with_song_name}'])
            song_list['spotifyName'].append(track['name'])
            song_list['uri'].append(track['uri'])
            
    return song_list

In [40]:
# applying uri funct to 2021
song_uri_df = pd.DataFrame(get_song_uri(song_titles_bb, 'song'))
# examining 2021 df w/ uris
song_uri_df.head()

,artist,song,spotifyName,uri
0,24kGoldn,Mood,Mood (feat. iann dior),spotify:track:4jPy3l0RUwlUI9T5XHBW2m
1,Pop Smoke,Mood,Mood Swings (feat. Lil Tjay),spotify:track:5rZlwNFl01HqLWBQGryKSm
2,24kGoldn,Mood,"Mood (Remix) feat. Justin Bieber, J Balvin & i...",spotify:track:6FuGJB290AQMAHTfjOYVaK
3,Koosen,Mood,Mood,spotify:track:7lTXqBURoqMMzzYA6fFF6s
4,Dustin Lynch,Mood,Small Town Boy,spotify:track:2YMhrXQYKkm4kXLcXKKd5z


In [42]:
song_uri_df.shape

(9999, 4)

In [43]:
# getting spotify song features for each song
features1 = {
    'song': [],
    'spotifyName': [],
    'artist': [],
    'features': []
}
features2 = {
    'song': [],
    'spotifyName': [],
    'artist': [],
    'features': []
}
# let code run till a break
for i, row in song_uri_df[:5000].iterrows():
    uri = row['uri']
    audio_features = spotify.audio_features(uri)[0]
    features1['song'].append(row['song'])
    features1['spotifyName'].append(row['spotifyName'])
    features1['artist'].append(row['artist'])
    features1['features'].append(audio_features)
    
for i, row in song_uri_df[5000:].iterrows():
    uri = row['uri']
    audio_features = spotify.audio_features(uri)[0]
    features2['song'].append(row['song'])
    features2['spotifyName'].append(row['spotifyName'])
    features2['artist'].append(row['artist'])
    features2['features'].append(audio_features)

In [44]:
# making the results of the api call into a df
m = pd.DataFrame(features1)
m.head()

,song,spotifyName,artist,features
0,Mood,Mood (feat. iann dior),24kGoldn,"{'danceability': 0.701, 'energy': 0.716, 'key'..."
1,Mood,Mood Swings (feat. Lil Tjay),Pop Smoke,"{'danceability': 0.48, 'energy': 0.602, 'key':..."
2,Mood,"Mood (Remix) feat. Justin Bieber, J Balvin & i...",24kGoldn,"{'danceability': 0.721, 'energy': 0.745, 'key'..."
3,Mood,Mood,Koosen,"{'danceability': 0.822, 'energy': 0.867, 'key'..."
4,Mood,Small Town Boy,Dustin Lynch,"{'danceability': 0.554, 'energy': 0.743, 'key'..."


In [45]:
# making the results of the api call into a df
n = pd.DataFrame(features2)
n.head()

,song,spotifyName,artist,features
0,What's Next,What’s Next,Drake,"{'danceability': 0.781, 'energy': 0.594, 'key'..."
1,What's Next,What's Next,Lacy B,"{'danceability': 0.83, 'energy': 0.541, 'key':..."
2,What's Next,What's Next,Slipknot,"{'danceability': 0.557, 'energy': 0.556, 'key'..."
3,What's Next,The Red,Chevelle,"{'danceability': 0.509, 'energy': 0.843, 'key'..."
4,Wants And Needs,Wants and Needs (feat. Lil Baby),Drake,"{'danceability': 0.578, 'energy': 0.449, 'key'..."


In [46]:
# renaming features output, renaming cols to fit with other dfs
bb_features = pd.concat([m,n])
bb_features = bb_features.rename(columns={'artist': 'performer'})
bb_features

,song,spotifyName,performer,features
0,Mood,Mood (feat. iann dior),24kGoldn,"{'danceability': 0.701, 'energy': 0.716, 'key'..."
1,Mood,Mood Swings (feat. Lil Tjay),Pop Smoke,"{'danceability': 0.48, 'energy': 0.602, 'key':..."
2,Mood,"Mood (Remix) feat. Justin Bieber, J Balvin & i...",24kGoldn,"{'danceability': 0.721, 'energy': 0.745, 'key'..."
3,Mood,Mood,Koosen,"{'danceability': 0.822, 'energy': 0.867, 'key'..."
4,Mood,Small Town Boy,Dustin Lynch,"{'danceability': 0.554, 'energy': 0.743, 'key'..."
...,...,...,...,...
4994,Go!,good 4 u,Olivia Rodrigo,"{'danceability': 0.556, 'energy': 0.661, 'key'..."
4995,Go!,Good Days,SZA,"{'danceability': 0.436, 'energy': 0.655, 'key'..."
4996,Go!,Goosebumps - Remix,Travis Scott,"{'danceability': 0.841, 'energy': 0.593, 'key'..."
4997,Go!,good 4 u,Olivia Rodrigo,"{'danceability': 0.563, 'energy': 0.664, 'key'..."


In [47]:
# expanding the features col into separate cols
feats = bb_features['features'].apply(pd.Series)

In [48]:
# recombining the features into the 2021 features df
bb_features_df = pd.concat([bb_features, feats], axis=1).drop('features', axis=1)
bb_features_df.head()

,song,spotifyName,performer,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Mood,Mood (feat. iann dior),24kGoldn,0.701,0.716,7.0,-3.671,0.0,0.0361,0.1740,...,0.3240,0.732,91.007,audio_features,4jPy3l0RUwlUI9T5XHBW2m,spotify:track:4jPy3l0RUwlUI9T5XHBW2m,https://api.spotify.com/v1/tracks/4jPy3l0RUwlU...,https://api.spotify.com/v1/audio-analysis/4jPy...,140533.0,4.0
1,Mood,Mood Swings (feat. Lil Tjay),Pop Smoke,0.480,0.602,10.0,-6.335,0.0,0.3680,0.4870,...,0.0848,0.269,179.798,audio_features,5rZlwNFl01HqLWBQGryKSm,spotify:track:5rZlwNFl01HqLWBQGryKSm,https://api.spotify.com/v1/tracks/5rZlwNFl01Hq...,https://api.spotify.com/v1/audio-analysis/5rZl...,213333.0,4.0
2,Mood,"Mood (Remix) feat. Justin Bieber, J Balvin & i...",24kGoldn,0.721,0.745,7.0,-3.508,0.0,0.0366,0.1510,...,0.3720,0.699,90.988,audio_features,6FuGJB290AQMAHTfjOYVaK,spotify:track:6FuGJB290AQMAHTfjOYVaK,https://api.spotify.com/v1/tracks/6FuGJB290AQM...,https://api.spotify.com/v1/audio-analysis/6FuG...,192745.0,4.0
3,Mood,Mood,Koosen,0.822,0.867,7.0,-4.478,0.0,0.2500,0.1830,...,0.1310,0.480,116.011,audio_features,7lTXqBURoqMMzzYA6fFF6s,spotify:track:7lTXqBURoqMMzzYA6fFF6s,https://api.spotify.com/v1/tracks/7lTXqBURoqMM...,https://api.spotify.com/v1/audio-analysis/7lTX...,144053.0,4.0
4,Mood,Small Town Boy,Dustin Lynch,0.554,0.743,9.0,-6.016,1.0,0.0287,0.0251,...,0.3590,0.673,72.997,audio_features,2YMhrXQYKkm4kXLcXKKd5z,spotify:track:2YMhrXQYKkm4kXLcXKKd5z,https://api.spotify.com/v1/tracks/2YMhrXQYKkm4...,https://api.spotify.com/v1/audio-analysis/2YMh...,205733.0,4.0


In [87]:
# preprocessing to ensure capitalisation does not impede song matches
# bb_features_df['song'] = [x.lower() for x in bb_features_df['song']]
# freshData['song'] = [x.lower() for x in freshData['song']]

<ipython-input-87-1d7f31614b58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freshData['song'] = [x.lower() for x in freshData['song']]


In [49]:
# merging the original 2021 df with the newly obtained track features
# mixing cols together on basis of same song title, so use pd.merge
fresh_merge = freshData.merge(bb_features_df, on='song', how='left', suffixes=('', '_drop'))
fresh_merge.drop([col for col in fresh_merge.columns if 'drop' in col], axis=1, inplace=True)
fresh_merge.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,spotifyName,danceability,energy,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,Mood (feat. iann dior),0.701,0.716,...,0.3240,0.732,91.007,audio_features,4jPy3l0RUwlUI9T5XHBW2m,spotify:track:4jPy3l0RUwlUI9T5XHBW2m,https://api.spotify.com/v1/tracks/4jPy3l0RUwlU...,https://api.spotify.com/v1/audio-analysis/4jPy...,140533.0,4.0
1,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,Mood Swings (feat. Lil Tjay),0.480,0.602,...,0.0848,0.269,179.798,audio_features,5rZlwNFl01HqLWBQGryKSm,spotify:track:5rZlwNFl01HqLWBQGryKSm,https://api.spotify.com/v1/tracks/5rZlwNFl01Hq...,https://api.spotify.com/v1/audio-analysis/5rZl...,213333.0,4.0
2,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,"Mood (Remix) feat. Justin Bieber, J Balvin & i...",0.721,0.745,...,0.3720,0.699,90.988,audio_features,6FuGJB290AQMAHTfjOYVaK,spotify:track:6FuGJB290AQMAHTfjOYVaK,https://api.spotify.com/v1/tracks/6FuGJB290AQM...,https://api.spotify.com/v1/audio-analysis/6FuG...,192745.0,4.0
3,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,Mood,0.822,0.867,...,0.1310,0.480,116.011,audio_features,7lTXqBURoqMMzzYA6fFF6s,spotify:track:7lTXqBURoqMMzzYA6fFF6s,https://api.spotify.com/v1/tracks/7lTXqBURoqMM...,https://api.spotify.com/v1/audio-analysis/7lTX...,144053.0,4.0
4,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,Small Town Boy,0.554,0.743,...,0.3590,0.673,72.997,audio_features,2YMhrXQYKkm4kXLcXKKd5z,spotify:track:2YMhrXQYKkm4kXLcXKKd5z,https://api.spotify.com/v1/tracks/2YMhrXQYKkm4...,https://api.spotify.com/v1/audio-analysis/2YMh...,205733.0,4.0


In [50]:
# removing and moving around cols
fresh_merge = fresh_merge[['song', 'performer', 'chart_position',
       'previous_position', 'peak', 'weeks_on_chart', 'hitTF', 
       'id', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 
       'duration_ms', 'time_signature']]
fresh_merge.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,4jPy3l0RUwlUI9T5XHBW2m,0.701,0.716,...,-3.671,0.0,0.0361,0.1740,0.000000,0.3240,0.732,91.007,140533.0,4.0
1,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,5rZlwNFl01HqLWBQGryKSm,0.480,0.602,...,-6.335,0.0,0.3680,0.4870,0.000000,0.0848,0.269,179.798,213333.0,4.0
2,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,6FuGJB290AQMAHTfjOYVaK,0.721,0.745,...,-3.508,0.0,0.0366,0.1510,0.000001,0.3720,0.699,90.988,192745.0,4.0
3,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,7lTXqBURoqMMzzYA6fFF6s,0.822,0.867,...,-4.478,0.0,0.2500,0.1830,0.000008,0.1310,0.480,116.011,144053.0,4.0
4,Mood,24kGoldn Featuring iann dior,1,7,1,21,1,2YMhrXQYKkm4kXLcXKKd5z,0.554,0.743,...,-6.016,1.0,0.0287,0.0251,0.000000,0.3590,0.673,72.997,205733.0,4.0


In [51]:
# replacing '-' in previous position cols with 0
fresh_merge['previous_position'] = fresh_merge['previous_position'].replace(['-'], 0)

In [52]:
# sorting for aid in removing duplicates
# using weeks on chart as proxy for date
fresh_merge_sorted = fresh_merge.sort_values(['song','weeks_on_chart'])
fresh_merge_sorted.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
82324,2 Much,Justin Bieber,68,0,68,1,1,0oaY19dUwZimIgzn3ZZLZO,0.583,0.444,...,-8.601,1.0,0.0456,0.5930,0.00000,0.353,0.167,119.590,152796.0,4.0
82325,2 Much,Justin Bieber,68,0,68,1,1,1gjmPHYd1SoEmsj8Rd15MC,0.583,0.444,...,-8.601,1.0,0.0456,0.5930,0.00000,0.353,0.167,119.590,152796.0,4.0
82326,2 Much,Justin Bieber,68,0,68,1,1,5WlsjnqQ8HNJmJDR6dYWdz,0.752,0.726,...,-4.590,0.0,0.0565,0.0197,0.00000,0.203,0.260,92.004,211520.0,4.0
82327,2 Much,Justin Bieber,68,0,68,1,1,6jtNk2zqbiZ9S7l6Gcp78A,0.642,0.933,...,-2.887,0.0,0.0871,0.0657,0.00001,0.087,0.158,150.039,182400.0,4.0
82328,2 Much,Justin Bieber,68,0,68,1,1,3WUb0cR9kC4axUKGXEQJx4,0.752,0.726,...,-4.590,0.0,0.0565,0.0197,0.00000,0.203,0.260,92.004,211520.0,4.0


In [86]:
# removing true duplicates (every bit of info the same)
fresh_merge_drops = fresh_merge_sorted.drop_duplicates(subset=['song', 'weeks_on_chart'], keep='first')
fresh_merge_drops.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
82324,2 Much,Justin Bieber,68,0,68,1,1,0oaY19dUwZimIgzn3ZZLZO,0.583,0.444,...,-8.601,1.0,0.0456,0.593,0.0,0.353,0.167,119.590,152796.0,4.0
5655,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,13,19,8,10,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1.0,0.0940,0.237,0.0,0.248,0.485,109.978,173711.0,4.0
11310,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,11,13,8,11,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1.0,0.0940,0.237,0.0,0.248,0.485,109.978,173711.0,4.0
16250,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,2,11,2,12,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1.0,0.0940,0.237,0.0,0.248,0.485,109.978,173711.0,4.0
22775,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,4,2,2,13,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1.0,0.0940,0.237,0.0,0.248,0.485,109.978,173711.0,4.0


In [87]:
# removing track duplicates (same song on different dates)
fresh_merge_doubledrops = fresh_merge_drops.drop_duplicates(subset=['song', 'performer'], keep='last')
fresh_merge_doubledrops.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
82324,2 Much,Justin Bieber,68,0,68,1,1,0oaY19dUwZimIgzn3ZZLZO,0.583,0.444,...,-8.601,1.0,0.0456,0.5930,0.000000,0.353,0.167,119.590,152796.0,4.0
1065,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,19,40,8,9,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1.0,0.0940,0.2370,0.000000,0.248,0.485,109.978,173711.0,4.0
126619,4 Da Gang,42 Dugg & Roddy Ricch,87,83,67,6,1,4KIkADzoVm3A10E9qyjsQ5,0.871,0.545,...,-10.489,1.0,0.1890,0.0143,0.000000,0.148,0.722,100.482,155720.0,4.0
31680,7 Summers,Morgan Wallen,35,23,6,24,1,4obHzpwGrjoTuZh2DItEMZ,0.466,0.786,...,-5.130,1.0,0.0328,0.4170,0.000061,0.125,0.571,203.903,210507.0,4.0
40895,865,Morgan Wallen,76,77,46,5,1,0sKEilrw1GrBlG6qsTCrP4,0.532,0.584,...,-5.254,1.0,0.0248,0.7230,0.000000,0.101,0.367,88.003,190680.0,4.0


In [88]:
# resetting index
fresh_merge_doubledrops = fresh_merge_doubledrops.reset_index(drop=True)
fresh_merge_doubledrops.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2 Much,Justin Bieber,68,0,68,1,1,0oaY19dUwZimIgzn3ZZLZO,0.583,0.444,...,-8.601,1.0,0.0456,0.5930,0.000000,0.353,0.167,119.590,152796.0,4.0
1,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,19,40,8,9,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1.0,0.0940,0.2370,0.000000,0.248,0.485,109.978,173711.0,4.0
2,4 Da Gang,42 Dugg & Roddy Ricch,87,83,67,6,1,4KIkADzoVm3A10E9qyjsQ5,0.871,0.545,...,-10.489,1.0,0.1890,0.0143,0.000000,0.148,0.722,100.482,155720.0,4.0
3,7 Summers,Morgan Wallen,35,23,6,24,1,4obHzpwGrjoTuZh2DItEMZ,0.466,0.786,...,-5.130,1.0,0.0328,0.4170,0.000061,0.125,0.571,203.903,210507.0,4.0
4,865,Morgan Wallen,76,77,46,5,1,0sKEilrw1GrBlG6qsTCrP4,0.532,0.584,...,-5.254,1.0,0.0248,0.7230,0.000000,0.101,0.367,88.003,190680.0,4.0


#### Last date

In [89]:
# getting last date recorded in the scrape
last_date_scraped = str(bb21_df['date'][len(bb21_df['date']) - 1]).split(' ')[0]
last_date_scraped

'2021-05-22'

In [90]:
# checking nulls
fresh_merge_doubledrops.isnull().sum()

song                 0
performer            0
chart_position       0
previous_position    0
peak                 0
weeks_on_chart       0
hitTF                0
id                   0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
duration_ms          0
time_signature       0
dtype: int64

In [91]:
# checking for id dups
fresh_merge_doubledrops[fresh_merge_doubledrops['id'].duplicated()==True]

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
69,Diamonds Dancing,Young Thug & Gunna Featuring Travis Scott,46,0,46,1,1,7uZqvqXzzYMKrexKxdcGmd,0.851,0.703,...,-7.351,0.0,0.303,0.00949,0.000026,0.0895,0.596,74.017,242600.0,4.0


In [92]:
# examining the dup values
fresh_merge_doubledrops[fresh_merge_doubledrops['id']=='7uZqvqXzzYMKrexKxdcGmd']

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
68,Diamonds,Sam Smith,59,49,39,20,1,7uZqvqXzzYMKrexKxdcGmd,0.851,0.703,...,-7.351,0.0,0.303,0.00949,0.000026,0.0895,0.596,74.017,242600.0,4.0
69,Diamonds Dancing,Young Thug & Gunna Featuring Travis Scott,46,0,46,1,1,7uZqvqXzzYMKrexKxdcGmd,0.851,0.703,...,-7.351,0.0,0.303,0.00949,0.000026,0.0895,0.596,74.017,242600.0,4.0


In [85]:
song_uri_df[song_uri_df['song']=='Diamonds']

,artist,song,spotifyName,uri
225,Young Stoner Life,Diamonds,Diamonds Dancing (feat. Travis Scott),spotify:track:7uZqvqXzzYMKrexKxdcGmd
226,Rihanna,Diamonds,Diamonds,spotify:track:1z9kQ14XBSN0r2v6fx4IdG
227,MARINA,Diamonds,Bubblegum Bitch,spotify:track:0ZFBKLOZLIM16RAUb5eomN
228,Sam Smith,Diamonds,Diamonds,spotify:track:5hvwx5i67IwnCkjl9VHkNv
229,Turnpike Troubadours,Diamonds,Diamonds & Gasoline,spotify:track:2nAHmQV9WKA6c1emQilxih
690,Young Stoner Life,Diamonds,Diamonds Dancing (feat. Travis Scott),spotify:track:7uZqvqXzzYMKrexKxdcGmd
691,Rihanna,Diamonds,Diamonds,spotify:track:1z9kQ14XBSN0r2v6fx4IdG
692,MARINA,Diamonds,Bubblegum Bitch,spotify:track:0ZFBKLOZLIM16RAUb5eomN
693,Sam Smith,Diamonds,Diamonds,spotify:track:5hvwx5i67IwnCkjl9VHkNv
694,Turnpike Troubadours,Diamonds,Diamonds & Gasoline,spotify:track:2nAHmQV9WKA6c1emQilxih


In [83]:
song_uri_df[song_uri_df['song']=='Diamonds Dancing']

,artist,song,spotifyName,uri
8224,Young Stoner Life,Diamonds Dancing,Diamonds Dancing (feat. Travis Scott),spotify:track:7uZqvqXzzYMKrexKxdcGmd
8225,Drake,Diamonds Dancing,Diamonds Dancing,spotify:track:2AGottAzfC8bHzF7kEJ3Wa
8226,Dsteez,Diamonds Dancing,Diamonds Dancing,spotify:track:0sq7ncjyHdQkGH9GDhuqLv
8227,Young Stoner Life,Diamonds Dancing,Diamonds Dancing (feat. Travis Scott),spotify:track:78c7BcbeDgKJjpBqmfrcBg
8228,Lil' Goldy,Diamonds Dancing,Diamonds Dancing,spotify:track:12SyPHD4zrVV8ReT2ZvnDk


In [97]:
bb_features_df[(bb_features_df['song']=='Diamonds')&(bb_features_df['performer']=='Sam Smith')].columns

Index(['song', 'spotifyName', 'performer', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href',
       'analysis_url', 'duration_ms', 'time_signature'],
      dtype='object')

In [105]:
gur = bb_features_df[(bb_features_df['song']=='Diamonds')&(bb_features_df['performer']=='Sam Smith')][['song', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'id','duration_ms', 'time_signature']]

In [114]:
gur.iloc[0, :]

song                              Diamonds
danceability                         0.653
energy                               0.667
key                                     10
loudness                            -6.764
mode                                     0
speechiness                         0.0423
acousticness                         0.154
instrumentalness                         0
liveness                             0.107
valence                              0.556
tempo                              104.181
id                  5hvwx5i67IwnCkjl9VHkNv
duration_ms                         213869
time_signature                           4
Name: 228, dtype: object

In [115]:
# finding true id for track
new_id = '5hvwx5i67IwnCkjl9VHkNv'
new_danceability = 0.653
new_energy = 0.667
new_key = 10
new_loudness= -6.764
new_mode = 0
new_speechiness = 0.0423
new_acousticness = 0.154
new_instrumentalness = 0
new_liveness = 0.107
new_valence = 0.556
new_tempo = 104.181
new_duration_ms = 213869
new_time_signature = 4   

fresh_merge_doubledrops.at[68,'id']=new_id
fresh_merge_doubledrops.at[68,'danceability']=new_danceability
fresh_merge_doubledrops.at[68,'energy']=new_energy
fresh_merge_doubledrops.at[68,'key']=new_key
fresh_merge_doubledrops.at[68,'loudness']=new_loudness
fresh_merge_doubledrops.at[68,'mode']=new_mode
fresh_merge_doubledrops.at[68,'speechiness']=new_speechiness
fresh_merge_doubledrops.at[68,'acousticness']=new_acousticness
fresh_merge_doubledrops.at[68,'instrumentalness']=new_instrumentalness
fresh_merge_doubledrops.at[68,'liveness']=new_liveness
fresh_merge_doubledrops.at[68,'valence']=new_valence
fresh_merge_doubledrops.at[68,'tempo']=new_tempo
fresh_merge_doubledrops.at[68,'duration_ms']=new_duration_ms
fresh_merge_doubledrops.at[68,'time_signature']=new_time_signature

In [116]:
fresh_merge_doubledrops[fresh_merge_doubledrops['id']=='5hvwx5i67IwnCkjl9VHkNv']

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
68,Diamonds,Sam Smith,59,49,39,20,1,5hvwx5i67IwnCkjl9VHkNv,0.653,0.667,...,-6.764,0.0,0.0423,0.154,0.0,0.107,0.556,104.181,213869.0,4.0


In [117]:
# rechecking dups
fresh_merge_doubledrops[fresh_merge_doubledrops['id'].duplicated()==True]

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [118]:
# exporting the renewed data to csv
fresh_merge_doubledrops.to_csv('../data/only2021.csv', index=False)